In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import random
import pandas as pd
from collections import defaultdict

In [ ]:
# Mission 2
# 라벨링 JSON 기반 통계 + 스타일 선호 정보표 생성
#경로
base_path = '/content/drive/MyDrive/fashion_data/sample'
label_root = os.path.join(base_path, 'labels_data')
image_root = os.path.join(base_path, 'raw_data')
splits = ['1.training', '2.validation']

# 유효 이미지 ID 수집
def extract_valid_image_ids():
    valid_ids = {}
    for split in splits:
        split_dir = os.path.join(image_root, split)
        ids = set()
        if not os.path.exists(split_dir):
            continue
        for gender in os.listdir(split_dir):
            gender_path = os.path.join(split_dir, gender)
            for year in os.listdir(gender_path):
                year_path = os.path.join(gender_path, year)
                for fname in os.listdir(year_path):
                    if fname.endswith('.jpg'):
                        prefix = fname.split('_')[0]
                        num = fname.split('_')[1]
                        ids.add(f"{prefix}_{num}")
        print(f"{split} 유효 이미지 수: {len(ids)}")
        valid_ids[split] = ids
    return valid_ids

# 2-1 성별,스타일 통계
def compute_gender_style_statistics(split, valid_ids):
    stats = defaultdict(set)
    label_split_dir = os.path.join(label_root, split)
    if not os.path.exists(label_split_dir):
        print(f"라벨 폴더 없음: {label_split_dir}")
        return pd.DataFrame()

    total_files = 0
    for gender in os.listdir(label_split_dir):
        gender_path = os.path.join(label_split_dir, gender)
        if not os.path.isdir(gender_path):
            continue
        for year in os.listdir(gender_path):
            year_path = os.path.join(gender_path, year)
            if not os.path.isdir(year_path):
                continue
            for fname in os.listdir(year_path):
                if not fname.endswith('.json'):
                    continue
                total_files += 1
                parts = fname.split('_')
                if len(parts) < 6:
                    continue
                img_id = f"{parts[0]}_{parts[1]}"
                style = parts[3]
                gender = parts[4]
                survey_id = parts[5].split('.')[0]
                if img_id in valid_ids:
                    stats[(gender, style)].add(survey_id)

    print(f"{split} 라벨 파일 수: {total_files}")

    df = pd.DataFrame([
        {'성별': g, '스타일': s, '응답 수': len(v)}
        for (g, s), v in sorted(stats.items(), key=lambda x: (-len(x[1]), x[0]))
    ])
    return df

# 실행 결과
valid_ids = extract_valid_image_ids()

for split in splits:
    print(f"\n {split} 성별 & 스타일 통계표")
    df_stat = compute_gender_style_statistics(split, valid_ids[split])
    if not df_stat.empty:
        display(df_stat.head(20))
    else:
        print("(통계 없음)")

1.training 유효 이미지 수: 1159
2.validation 유효 이미지 수: 1220

 1.training 성별 & 스타일 통계표
1.training 라벨 파일 수: 180696


,성별,스타일,응답 수
0,M,hippie,453
1,M,ivy,407
2,W,bodyconscious,317
3,W,minimal,279
4,M,hiphop,234
5,M,metrosexual,233
6,W,feminine,221
7,M,mods,206
8,M,bold,192
9,W,classic,169



 2.validation 성별 & 스타일 통계표
2.validation 라벨 파일 수: 32144


,성별,스타일,응답 수
0,M,ivy,414
1,M,hippie,390
2,M,mods,294
3,W,feminine,286
4,M,metrosexual,241
5,M,bold,240
6,M,hiphop,214
7,W,powersuit,198
8,W,classic,178
9,M,sportivecasual,174


In [ ]:
# 2 -2 (응답자 스타일 선호 정보표)
base_path = '/content/drive/MyDrive/fashion_data/sample'
label_root = os.path.join(base_path, 'labels_data')
split = '2.validation'

def extract_valid_image_ids(split):
    image_root = os.path.join(base_path, 'raw_data')
    split_dir = os.path.join(image_root, split)
    ids = set()
    for gender in os.listdir(split_dir):
        gender_path = os.path.join(split_dir, gender)
        for year in os.listdir(gender_path):
            year_path = os.path.join(gender_path, year)
            for fname in os.listdir(year_path):
                if fname.endswith('.jpg'):
                    parts = fname.split('_')
                    if len(parts) >= 2:
                        prefix = parts[0]  # W or T
                        num = parts[1]     # 숫자
                        ids.add(f"{prefix}_{num}")
    print(f"{split} 유효 이미지 수: {len(ids)}")
    return ids

#  스타일 선호 표 출력
def build_preference_table(split, valid_ids, sample_limit=300):
    label_dir = os.path.join(label_root, split)
    table = defaultdict(dict)
    total = 0

    for gender in os.listdir(label_dir):
        gender_path = os.path.join(label_dir, gender)
        for year in os.listdir(gender_path):
            year_path = os.path.join(gender_path, year)
            all_jsons = [f for f in os.listdir(year_path) if f.endswith('.json')]
            sampled = random.sample(all_jsons, min(sample_limit, len(all_jsons)))

            for fname in sampled:
                parts = fname.split('_')
                if len(parts) < 6:
                    continue
                img_id = f"{parts[0]}_{parts[1]}"
                if img_id not in valid_ids:
                    continue
                try:
                    with open(os.path.join(year_path, fname), encoding='utf-8') as f:
                        data = json.load(f)
                    user_id = data.get('user', {}).get('R_id')
                    q5 = data.get('item', {}).get('survey', {}).get('Q5')
                    if user_id and q5 in ['1', '2']:
                        table[user_id][fname] = int(q5)
                        total += 1
                except:
                    continue

    print(f"총 스타일 선호 응답 수: {total}")
    df = pd.DataFrame(table).T.fillna(0).astype(int)
    return df

#실행 결과
valid_ids = extract_valid_image_ids(split)
preference_df = build_preference_table(split, valid_ids)

print("\n 2-2: 스타일 선호 정보표 (validation 기준)")
display(preference_df.head())

2.validation 유효 이미지 수: 1220
총 스타일 선호 응답 수: 0

 2-2: 스타일 선호 정보표 (validation 기준, 간소화 버전)


""
